In [0]:
%run "./includes/configuration"

#### Load silver layer data 

In [0]:
supplier_df = spark.read.format('delta').load(f'{silver_folder_path}/supplier')
product_df = spark.read.format('delta').load(f'{silver_folder_path}/product')
transportation_df = spark.read.format('delta').load(f'{silver_folder_path}/transportation')
purchaseorder_df = spark.read.format('delta').load(f'{silver_folder_path}/purchaseorder')

In [0]:
display(supplier_df.limit(5))

S_ID,Tin_No,Company_Name,Date_of_Reg,SubCity,Town,Telephone,Fax,EMail,Business_License_No,Business_Type,Negotiation_Score,Defect_Quality
1.0,Local-0004765279,ßììßëàßê¡ ßîáßëàßêïßêï ßèòßîìßï╡ Fiker General Trading,21-03-2019,Akaki Kality,Addis Ababa,2.51911E+11,2.51114E+11,betebebubi@yahoo.com,14/665/1910/2005,Computers and Related Equipments,0.75,Low
2.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653400/2009,Chemicals and chemical products,0.75,Low
3.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653407/2009,Laboratory materials and Equipment,0.75,Low
4.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,03/213/01/3025/2005,Grade VI,0.75,Low
5.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,AM/BD/HDR/03/213/6011912/2008,Building Materials,0.75,Low


In [0]:
display(product_df.limit(5))

P_ID,Product_type,Price,Price_Category
SKU0,Iron Materials,69.80800554,Medium Price
SKU1,Copper Materials,14.84352328,Low Price
SKU2,Iron Materials,11.31968329,Low Price
SKU3,Copper Materials,61.16334302,Medium Price
SKU4,Copper Materials,4.805496036,Low Price


In [0]:
display(transportation_df)

TRANSPORTATION_ID,MODES,Priority
1.0,Road,0
2.0,Air,0
3.0,Rail,0
4.0,Sea,0


In [0]:
display(purchaseorder_df)

PO_ID,P_ID,Availability,Number_of_products_sold,Revenue_generated,Stock_levels,Lead_times,Order_quantities,Shipping_times,Shipping_carriers,Shipping_costs,Supplier_Id,Location,Lead_time,Production_volumes,Manufacturing_lead_time,Manufacturing_costs,Inspection_results,Defect_rates,Transportation_id,Routes,Costs,Total_Cost
96,SKU95,65.0,672.0,7386.363944,15.0,14.0,26.0,9.0,Carrier B,8.63033887,26504.0,Mumbai,18.0,450.0,26.0,58.89068577,Pending,1.21088213,2.0,Route A,778.8642414,20250.4702764
97,SKU96,29.0,324.0,7698.424766,67.0,2.0,32.0,3.0,Carrier C,5.352878044,5530.0,Mumbai,28.0,648.0,28.0,17.80375633,Pending,3.872047681,1.0,Route A,188.7421411,6039.7485152
98,SKU97,56.0,62.0,4370.91658,46.0,19.0,4.0,9.0,Carrier A,7.904845611,247.0,Mumbai,10.0,535.0,13.0,65.76515593,Fail,3.376237835,1.0,Route A,540.1324229,2160.5296916
99,SKU98,43.0,913.0,8525.95256,53.0,1.0,27.0,7.0,Carrier B,1.409801095,31074.0,Chennai,28.0,581.0,9.0,5.604690864,Pending,2.908122169,3.0,Route A,882.1988635,23819.3693145
100,SKU99,17.0,627.0,9185.185829,55.0,8.0,59.0,6.0,Carrier B,1.311023756,33277.0,Chennai,29.0,921.0,2.0,38.07289852,Fail,0.346027291,3.0,Route B,210.743009,12433.837531


#### Rank suppliers based on total no of services they provide

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, col, count_distinct, dense_rank, sum, avg, coalesce, lit

In [0]:
window_spec = Window.orderBy(desc('Service_Count'))

ranked_supplier = supplier_df.groupBy(col('Company_Name')) \
                            .agg(count_distinct(col('Business_Type')).alias('Service_Count')) \
                            .withColumn('Rank', dense_rank().over(window_spec))

display(ranked_supplier.limit(5))

Company_Name,Service_Count,Rank
FAREX TRADING Co. Ltd.,24,1
Robel alemu,20,2
Gashaw Bishaw Negash,17,3
Hoshi Alliance Trading PLC,15,4
Akber Industtry PLC,15,4


#### Write ranked supplier data to gold container in delta format

In [0]:
ranked_supplier.write.mode('overwrite').format('delta').save(f'{gold_folder_path}/supplier_ranked_view')

#### Recommend supplier based on their supplying data

In [0]:
# Define a list of columns to handle non-numeric values and cast to double
numeric_columns = ["Number_of_products_sold", "Revenue_generated", "Stock_levels", "Lead_times",
                   "Order_quantities", "Shipping_times", "Shipping_costs", "Total_Cost"]

for column in numeric_columns:
    purchaseorder_df = purchaseorder_df.withColumn(column, col(column).cast('double'))


# Aggregate the purchaseorder data based on on each unique supplier
purchaseorder_agg_df = purchaseorder_df.groupBy('Supplier_Id').agg(
    count_distinct('PO_ID').alias('Distinct_PurchaseOrder_Count'),
    sum('Number_of_products_sold').alias('Total_Products_Sold'),
    sum('Revenue_generated').alias('Total_Revenue'),
    avg('Stock_levels').alias('Avg_Stock_levels'),
    avg('Lead_times').alias('Avg_Lead_Times'),
    sum('Order_quantities').alias('Total_Order_Quantities'),
    avg('Shipping_times').alias('Avg_Shipping_Times'),
    count_distinct('Shipping_carriers').alias('Distinct_Shipping_Carriers'),
    avg('Shipping_costs').alias('Avg_Shipping_Costs'),
    sum('Total_Cost').alias('Total_Cost')
)


# Join this agg data with supplier data
joined_df = purchaseorder_agg_df.join(supplier_df, col('S_ID') == col('Supplier_Id'))

# Sum-up all the aggregated data; that denotes the recommendation_score! Higher the score, higher the recommendation
recomd_supply_df = joined_df.withColumn('Recommendation_Score',        
            coalesce(col("Total_Products_Sold").cast("double"), lit(0)) +
            coalesce(col("Total_Revenue").cast("double"), lit(0)) +
            coalesce(col("Avg_Stock_Levels").cast("double"), lit(0)) + 
            coalesce(col("Avg_Lead_Times").cast("double"), lit(0)) +
            coalesce(col("Total_Order_Quantities").cast("double"), lit(0)) + 
            coalesce(col("Avg_Shipping_Times").cast("double"), lit(0)) +
            coalesce(col("Distinct_Shipping_Carriers").cast("double"), lit(0)) + 
            coalesce(col("Avg_Shipping_Costs").cast("double"), lit(0)) + 
            coalesce(col("Total_Cost").cast("double"), lit(0)) +
            coalesce(col("Negotiation_Score").cast("double"), lit(0)) +
            coalesce(col("Defect_Quality").cast("double"), lit(0))
        )

# Rank recommendation score
window_spec = Window.orderBy(desc('Recommendation_Score'))
final_df = recomd_supply_df.withColumn('Rank', dense_rank().over(window_spec))

In [0]:
display(purchaseorder_agg_df)

Supplier_Id,Distinct_PurchaseOrder_Count,Total_Products_Sold,Total_Revenue,Avg_Stock_levels,Avg_Lead_Times,Total_Order_Quantities,Avg_Shipping_Times,Distinct_Shipping_Carriers,Avg_Shipping_Costs,Total_Cost
26504.0,1,672.0,7386.363944,15.0,14.0,26.0,9.0,1,8.63033887,20250.4702764
31074.0,1,913.0,8525.95256,53.0,1.0,27.0,7.0,1,1.409801095,23819.3693145
33277.0,1,627.0,9185.185829,55.0,8.0,59.0,6.0,1,1.311023756,12433.837531
247.0,1,62.0,4370.91658,46.0,19.0,4.0,9.0,1,7.904845611,2160.5296916
5530.0,1,324.0,7698.424766,67.0,2.0,32.0,3.0,1,5.352878044,6039.7485152


In [0]:
display(final_df)

Supplier_Id,Distinct_PurchaseOrder_Count,Total_Products_Sold,Total_Revenue,Avg_Stock_levels,Avg_Lead_Times,Total_Order_Quantities,Avg_Shipping_Times,Distinct_Shipping_Carriers,Avg_Shipping_Costs,Total_Cost,S_ID,Tin_No,Company_Name,Date_of_Reg,SubCity,Town,Telephone,Fax,EMail,Business_License_No,Business_Type,Negotiation_Score,Defect_Quality,Recommendation_Score,Rank
31074.0,1,913.0,8525.95256,53.0,1.0,27.0,7.0,1,1.409801095,23819.3693145,31074.0,Local-0007556445,ASHKID TRANSIT & LOGISTICS SERVICE,01-03-2017,BOLE,ADDIS ABEBA,+251 911 166477,Unknown,ashkidtransit@gmail.com,14/673/123130/2005,Customs Clearance and Fright Forwards,0.75,Low,33349.481675595,1
26504.0,1,672.0,7386.363944,15.0,14.0,26.0,9.0,1,8.63033887,20250.4702764,26504.0,Local-0046089252,Debresalem Consultancy and promotion PL.C,06-12-2016,Addis Ababa,Addis Ababa,0911-604989,Unknown,asanahom@yahoo.com,BL/AA/14/673/4911173/2007,Stationary Materials and Sanitary Items,0.75,Low,28383.21455927,2
33277.0,1,627.0,9185.185829,55.0,8.0,59.0,6.0,1,1.311023756,12433.837531,33277.0,Local-0048045147,AGATON BUSINESS PLC,04-02-2016,KIRKOS,ADDIS ABABA,+251 0966215698,Unknown,agaton002016@gmail.com,MT/AA/14/706/535837/2008,Machinery Equipment and Supplies,0.75,Low,22377.084383756,3
5530.0,1,324.0,7698.424766,67.0,2.0,32.0,3.0,1,5.352878044,6039.7485152,5530.0,Local-0027232227,TEMESGEN KASSYE,06-03-2015,bole,addis abeba,911712683,Unknown,yajosey@hotmail.com,temesgen kasye general con finishing work,Grade VIII(8),0.75,Low,14173.276159244,4
247.0,1,62.0,4370.91658,46.0,19.0,4.0,9.0,1,7.904845611,2160.5296916,247.0,Local-0042413611,Zewdu Tefaye Woldeyes,18-10-2019,ARADA,ADDIS ABABA,911650831,Unknown,zewzawe@gmail.com,KK/AA/14/706/772497/2010,Architecture and Engineering design,0.75,Low,6681.101117211001,5


#### Write recommended supplier data to gold container in delta format

In [0]:
final_df.write.mode('overwrite').format('delta').save(f'{gold_folder_path}/supplier_recomd_view')

#### Cleaning Old data

In [0]:
from delta.tables import DeltaTable

supplier_recomd_dt = DeltaTable.forPath(spark, f'{gold_folder_path}/supplier_recomd_view')
supplier_ranked_dt = DeltaTable.forPath(spark, f'{gold_folder_path}/supplier_ranked_view')

supplier_recomd_dt.vacuum(retentionHours=24)
supplier_ranked_dt.vacuum(retentionHours=24)